In [1]:
import os
import json
import urllib.request
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

import sys
sys.path.append("..")
from models.bounding_box import FeatureType, Point, BoundingBox, DSU
from utils.cv_preprocess import draw_boxes, group_bounding_boxes, merge_box_groups

from utils.file_utils import prepare_image_local, prepare_image_web, load_counter, save_counter, save_json

from utils.nlp_preprocess import load_spacy_models, is_english, is_chinese, split_dish_info, is_word_relevant
from models.word_unit import WordUnit
from models.dish_segmenter import Dish
from IPython.display import display

In [2]:
driver_path = r"F:\Fork_git\Labelling_Menu_Data\menu_scraper\webdriver\chromedriver\chromedriver.exe"


In [3]:
# Function to check if a string contains Chinese characters
def contains_chinese(text):
    return any('\u4e00' <= char <= '\u9fff' for char in text)


In [4]:
# Function to load JSON data from file
def load_json(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        return []


# File to store JSON data
json_file = './uber_london_dishes_data.json'

# Load existing data if available
restaurant_dishes = load_json(json_file)

In [5]:
import re
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains


def close_popup(driver):
    try:
        # Wait for the popup to be visible and get the close button
        wait = WebDriverWait(driver, 5)
        close_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-testid='close-button']")))
        close_button.click()
    except (NoSuchElementException, TimeoutException):
        # If no popup is found or it doesn't appear within 10 seconds, ignore
        pass
 
# Setup Chrome driver
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)





In [7]:
# File to store JSON data
json_file = './uber_london_dishes_data.json'

# Load existing data if available
restaurant_links = load_json('./uber_restaurant_links.json')

In [8]:


# Find all elements that match the 'store-card' data-testid attribute
# restaurant_elements = driver.find_elements(By.XPATH, "//a[@data-testid='store-card']")

# Extract the href attributes (links) from the elements
# restaurant_links = [element.get_attribute('href') for element in restaurant_elements]




# close_popup(driver)

# List to store dish information
# restaurant_dishes = []

# Go through each restaurant link
while restaurant_links:
    link = restaurant_links.pop(0)
    # Open the restaurant link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(5)
    close_popup(driver)
    
    # Find all dish elements
    dish_elements = driver.find_elements(By.XPATH, "//li[starts-with(@data-test, 'store-item')]")

    

    for dish_element  in dish_elements:
        name = ""
        description = ""

        try:
            # Attempt to find the name element
            name_element = dish_element.find_element(By.XPATH, ".//div[contains(@class,'be ct bg du b1')]/span")
            if name_element:
                name = name_element.text
        except NoSuchElementException:
            # If the name element is not found, the name remains "No Name"
            pass

        # Check if the name contains both Chinese and English
        if contains_chinese(name) and re.search("[a-zA-Z]", name):
            # Extract the dish description
            try:
                # Attempt to find the description element
                description_element = dish_element.find_element(By.XPATH, ".//div[contains(@class,'in h2 j2 h3 bl j3 cm')]/div/span")
                if description_element:
                    description = description_element.text
                    

            except NoSuchElementException:
                # If the description element is not found, just continue
                pass

        
        if name and contains_chinese(name) and re.search("[a-zA-Z]", name):
            restaurant_dishes.append({"name": name, "description": description})

            # Save updated data to JSON file
    
    save_json(restaurant_dishes, json_file)
    print("Unfinished links")
    print(restaurant_links)


    # # Output the extracted information for the current restaurant
    # print(f"Dishes from {link}:")
    # for dish in dishes:
    #     print(dish)



File has been saved to ./uber_london_dishes_data.json
Unfinished links
['https://www.ubereats.com/gb/store/new-ming/y_ugB_ThRhimuatoZAHfaA?diningMode=PICKUP', 'https://www.ubereats.com/gb/store/jaggis/Zj0XhkSZTbi9VbT6LcB51g?diningMode=PICKUP', 'https://www.ubereats.com/gb/store/paya/hkYtUQQgSQS_bPTK_r9SJg?diningMode=PICKUP', 'https://www.ubereats.com/gb/store/crispy-dosa-woodgreen/Os3KFVtMThGnD7DEgCthXw?diningMode=PICKUP', 'https://www.ubereats.com/gb/store/mumbai-dosa/vVbCQt5UTImq7jUGUToMqw?diningMode=PICKUP', 'https://www.ubereats.com/gb/store/chini-chor/hVR57S76SEKpc8CbKcexkA?diningMode=PICKUP', 'https://www.ubereats.com/gb/store/the-maharaja-sweets-%26-savouries-harrow/PqKhcGl2RbycA-M5S8-_Xg?diningMode=PICKUP', 'https://www.ubereats.com/gb/store/taste-of-china/GCPveaC2QduMjLZw5tGIdw?diningMode=PICKUP', 'https://www.ubereats.com/gb/store/wokabout/3gx7BDF1QB2BtJ9F7mi0vQ?diningMode=PICKUP', 'https://www.ubereats.com/gb/store/man-ho-restaurant/pDi9_toORGqAKzIOQYwajA?diningMode=PICKUP',

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=121.0.6167.86)
Stacktrace:
	GetHandleVerifier [0x00007FF673A45E42+3538674]
	(No symbol) [0x00007FF673664C02]
	(No symbol) [0x00007FF673515AEB]
	(No symbol) [0x00007FF673502FE8]
	(No symbol) [0x00007FF673502E9F]
	(No symbol) [0x00007FF6735013A5]
	(No symbol) [0x00007FF673501E5F]
	(No symbol) [0x00007FF67350F32E]
	(No symbol) [0x00007FF673521D83]
	(No symbol) [0x00007FF6735261DA]
	(No symbol) [0x00007FF673502420]
	(No symbol) [0x00007FF673521B9A]
	(No symbol) [0x00007FF67359BCBA]
	(No symbol) [0x00007FF67357EE53]
	(No symbol) [0x00007FF67354F514]
	(No symbol) [0x00007FF673550631]
	GetHandleVerifier [0x00007FF673A76CAD+3738973]
	GetHandleVerifier [0x00007FF673ACC506+4089270]
	GetHandleVerifier [0x00007FF673AC4823+4057299]
	GetHandleVerifier [0x00007FF673795C49+720121]
	(No symbol) [0x00007FF67367126F]
	(No symbol) [0x00007FF67366C304]
	(No symbol) [0x00007FF67366C432]
	(No symbol

In [ ]:
dish_elements = driver.find_elements(By.XPATH, "//li[starts-with(@data-test, 'store-item')]")

In [ ]:
# for dish_element  in dish_elements:
#     name = ""
#     description = ""

#     try:
#         # Attempt to find the name element
#         name_element = dish_element.find_element(By.XPATH, ".//div[contains(@class,'be ct bg du b1')]/span")
#         if name_element:
#             name = name_element.text
#     except NoSuchElementException:
#         # If the name element is not found, the name remains "No Name"
#         pass

#     # Check if the name contains both Chinese and English
#     if contains_chinese(name) and re.search("[a-zA-Z]", name):
#         # Extract the dish description
#         try:
#             # Attempt to find the description element
#             description_element = dish_element.find_element(By.XPATH, ".//div[contains(@class,'iq h6 ko h7 bl kp cm')]/div/span")
#             if description_element:
#                 description = description_element.text
                

#         except NoSuchElementException:
#             # If the description element is not found, just continue
#             pass
#     print(name, description)

Brown Sugar Biscuits 黑糖餅 Traditional Taiwanese biscuits, authentic taste to its finest
Pearl Milk Tea 珍珠奶茶 
Taro Bobo Milk Tea 芋泥波波茶 
Panda Milk Tea 熊貓奶茶 
Oreo Potted Milk Tea 盆栽奶茶 
QQ Style Milk Tea 雙Q奶茶 
Grass Jelly Milk Tea 嫩仙草奶茶 
Fresh Taro Milk Tea 鮮芋奶茶 
French Pudding Milk Tea 法式布丁奶 
Crystal Milk Tea 水晶奶茶 
Milk Tea with Coconut Jelly 椰果奶茶 
Red Bean Milk Tea 紅豆奶茶 
Milk Tea with Coffee Jelly 咖啡凍奶茶 
🆕 Mango Fanatics 楊枝甘露 
Taro Milk Tea 香芋奶茶 
Earl Grey Milk Tea 伯爵奶茶 
Classic Rose Milk Tea 古典玫瑰奶 
Jasmine Green Milk Tea 茉香奶綠 
Matcha Latte (nv) 京都抹茶拿鐵 
Roasted Oolong Milk Tea 炭培烏龍奶 
Jadeite Royal Milk Tea 翠玉奶茶 
Ballet Chocolate Milk Tea 芭蕾可可 
🆕 Mulberry Milk Tea 桑葚奶茶 
Crown Chocolate 奶蓋可可 
Crown Earl Grey Tea 奶蓋紅茶 
Crown Oolong Tea 奶蓋烏龍 
Crown Jadeite Royal Tea 奶蓋清茶 
Crown Green Tea 奶蓋綠茶 
Cream Double Green 茶凍奶蓋綠 
Passion Fruit Royal Tea 百香清茶 
Mango Royal Tea 芒果清茶 
Honey Peach Royal Tea 水蜜桃清茶 
Lychee Black Tea with Aloe貴妃蜜紅茶 
Elegant Rose Royal Tea 玫瑰清茶 
Lemon Royal Tea 檸檬清茶 
Strawberry

In [ ]:
dish_elements

[]

In [ ]:
# Close the driver
driver.quit()